In [1]:
import time
import pandas as pd
import requests

# ------------------------------------------------------------
# Konfiguration
# ------------------------------------------------------------
BASE_URL = "https://evetycoon.com/api/v1/market/orders/"
HEADERS = {"User-Agent": "EVE-Tycoon-Analyzer/1.0"}

# Minerale (offizielle typeIDs)
all_items = [
    {"typeID": 34, "typeName": "Tritanium"},
    {"typeID": 35, "typeName": "Pyerite"},
    {"typeID": 36, "typeName": "Mexallon"},
    {"typeID": 37, "typeName": "Isogen"},
    {"typeID": 38, "typeName": "Nocxium"},
    {"typeID": 39, "typeName": "Zydrine"},
    {"typeID": 40, "typeName": "Megacyte"},
    {"typeID": 11399, "typeName": "Morphite"}
]

# ------------------------------------------------------------
# Hilfsfunktionen
# ------------------------------------------------------------
def get_orders_for_item(type_id, type_name):
    """Holt Marktdaten für ein Item über die EVE Tycoon API"""
    url = f"{BASE_URL}{type_id}"
    r = requests.get(url, headers=HEADERS, timeout=20)
    if r.status_code != 200:
        print(f"⚠️ Fehler für {type_name}: {r.status_code}")
        return None
    return r.json()


def _label_row(system_name, region_name, station_name, structure_name):
    place = station_name if station_name else structure_name
    return f"{place} ({system_name}, {region_name})"


# ------------------------------------------------------------
# Hauptlogik
# ------------------------------------------------------------
print("\n📦 Lade ALLE Sell-Orders für Mineralien...")

all_sell_rows = []

for idx, item in enumerate(all_items, 1):
    type_id = item["typeID"]
    type_name = item["typeName"]
    print(f"[{idx}/{len(all_items)}] → Abruf: {type_name}")

    data = get_orders_for_item(type_id, type_name)
    if not data or "orders" not in data:
        continue

    stations = data.get("stationNames", {})
    structures = data.get("structureNames", {})
    systems = data.get("systems", {})

    sells = [o for o in data["orders"] if not o.get("isBuyOrder", False) and o.get("volumeRemain", 0) > 0]
    if not sells:
        continue

    # --- ✅ pro Station nur die günstigste Order + Volumen ---
    df_temp = pd.DataFrame(sells)
    df_temp["locationId"] = df_temp["locationId"].astype(str)
    df_temp = (
        df_temp.sort_values(["locationId", "price"])
        .groupby("locationId", as_index=False)
        .first()[["locationId", "price", "volumeRemain", "regionId", "systemId"]]
    )

    for _, row in df_temp.iterrows():
        loc_id = row["locationId"]
        sys_id = str(row["systemId"])
        region_name = systems.get(sys_id, {}).get("regionName", row["regionId"])
        system_name = systems.get(sys_id, {}).get("solarSystemName", sys_id)
        station_name = stations.get(loc_id, "")
        structure_name = structures.get(loc_id, "")

        all_sell_rows.append({
            "typeID": type_id,
            "typeName": type_name,
            "locationId": loc_id,
            "price": float(row["price"]),
            "volumeRemain": int(row["volumeRemain"]),
            "regionName": region_name,
            "systemName": system_name,
            "stationName": station_name,
            "structureName": structure_name,
            "Ort": _label_row(system_name, region_name, station_name, structure_name)
        })

    # API bitte nicht überlasten
    time.sleep(0.3)

df_sells_all = pd.DataFrame(all_sell_rows)
if df_sells_all.empty:
    raise RuntimeError("❌ Keine Sell-Orders gefunden.")
print(f"✅ Sell-Orders (bereinigt, pro Station günstigste Order): {len(df_sells_all):,}")

# ------------------------------------------------------------
# Aggregation: niedrigster Preis je Station
# ------------------------------------------------------------
station_min_prices = (
    df_sells_all
    .groupby(["typeName", "locationId"], as_index=False)
    .agg(
        min_price=("price", "min"),
        volumeRemain=("volumeRemain", "first"),
        regionName=("regionName", "first"),
        systemName=("systemName", "first"),
        stationName=("stationName", "first"),
        structureName=("structureName", "first"),
        Ort=("Ort", "first")
    )
)

# ------------------------------------------------------------
# Für jedes Mineral:
# global günstigster Preis (Einkauf)
# günstigster Preis in Jita IV – Moon 4 – Caldari Navy Assembly Plant (Verkauf)
# ------------------------------------------------------------
records = []

for item_name, g in station_min_prices.groupby("typeName"):
    g["stationName"] = g["stationName"].astype(str).fillna("")
    g["systemName"] = g["systemName"].astype(str).fillna("")
    g["regionName"] = g["regionName"].astype(str).fillna("")

    # 1️⃣ global günstigster Preis (Einkauf)
    buy_row = g.loc[g["min_price"].idxmin()]
    buy_price = buy_row["min_price"]
    buy_loc = buy_row["Ort"]
    buy_vol = int(buy_row["volumeRemain"])

    # 2️⃣ günstigster Preis in Jita IV – Moon 4 – Caldari Navy Assembly Plant
    jita_rows = g[g["stationName"].str.lower().eq("jita iv - moon 4 - caldari navy assembly plant")]
    if jita_rows.empty:
        continue

    jita_row = jita_rows.loc[jita_rows["min_price"].idxmin()]
    jita_price = jita_row["min_price"]
    jita_loc = jita_row["Ort"]

    # 3️⃣ Spread berechnen
    spread_abs = jita_price - buy_price
    spread_pct = (spread_abs / buy_price) * 100 if buy_price > 0 else 0

    records.append({
        "Item": item_name,
        "Kaufpreis [ISK]": buy_price,
        "Kauf-Ort": buy_loc,
        "Verfügbares Volumen (Kauf)": buy_vol,
        "Jita-Verkaufspreis [ISK]": jita_price,
        "Jita-Ort": jita_loc,
        "Spread [ISK]": spread_abs,
        "Spread [%]": spread_pct
    })

df_result = (
    pd.DataFrame(records)
    .sort_values("Spread [%]", ascending=False)
    .reset_index(drop=True)
)

# ------------------------------------------------------------
# Anzeige
# ------------------------------------------------------------
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 220)

print("\n📈 Günstig kaufen → In Jita IV – Moon 4 – Caldari Navy Assembly Plant verkaufen:\n")
display(df_result)



📦 Lade ALLE Sell-Orders für Mineralien...
[1/8] → Abruf: Tritanium
[2/8] → Abruf: Pyerite
[3/8] → Abruf: Mexallon
[4/8] → Abruf: Isogen
[5/8] → Abruf: Nocxium
[6/8] → Abruf: Zydrine
[7/8] → Abruf: Megacyte
[8/8] → Abruf: Morphite
✅ Sell-Orders (bereinigt, pro Station günstigste Order): 1,686

📈 Günstig kaufen → In Jita IV – Moon 4 – Caldari Navy Assembly Plant verkaufen:



,Item,Kaufpreis [ISK],Kauf-Ort,Verfügbares Volumen (Kauf),Jita-Verkaufspreis [ISK],Jita-Ort,Spread [ISK],Spread [%]
0,Pyerite,1.47,Nisuwa VII - State Protectorate Logistic Suppo...,927,34.59,Jita IV - Moon 4 - Caldari Navy Assembly Plant...,33.12,"2,253.06"
1,Nocxium,113.60,Muvolailen VIII - Rapid Assembly Factory (Muvo...,200,"1,194.00",Jita IV - Moon 4 - Caldari Navy Assembly Plant...,"1,080.40",951.06
2,Tritanium,0.74,Uplingur II - Moon 1 - Nefantar Miner Associat...,391,4.21,Jita IV - Moon 4 - Caldari Navy Assembly Plant...,3.47,468.92
3,Mexallon,35.05,Nisuwa VII - State Protectorate Logistic Suppo...,746,65.73,Jita IV - Moon 4 - Caldari Navy Assembly Plant...,30.68,87.53
4,Megacyte,"2,701.00",Sasta VI - Moon 3 - Ammatar Fleet Assembly Pla...,1,"3,569.00",Jita IV - Moon 4 - Caldari Navy Assembly Plant...,868.00,32.14
5,Isogen,240.10,RV5-DW VII - Moon 2 - True Power Assembly Plan...,5199,310.00,Jita IV - Moon 4 - Caldari Navy Assembly Plant...,69.90,29.11
6,Zydrine,"1,285.00",Aufay III - Moon 3 - Astral Mining Inc. Mining...,2,"1,640.00",Jita IV - Moon 4 - Caldari Navy Assembly Plant...,355.00,27.63
7,Morphite,"22,390.00",Jita IV - Moon 4 - Caldari Navy Assembly Plant...,48270,"22,390.00",Jita IV - Moon 4 - Caldari Navy Assembly Plant...,0.00,0.00
